# Funkcje skrótu

## Wstęp

W tym laboratorium zapoznamy się z algorytmem funkcji skrótu MD5, czyli jednym z klasycznych przykładów funkcji mieszających (hashujących), które były powszechnie stosowane w kryptografii, ochronie danych oraz weryfikacji integralności. Funkcja MD5 generuje 128-bitowy skrót (hash) z dowolnego wejścia tekstowego, zapewniając, że wynik jest zawsze tej samej długości, niezależnie od wielkości danych wejściowych.

Funkcje skrótu znajdują szerokie zastosowanie w informatyce, m.in.:
- przy weryfikacji integralności danych,
- w przechowywaniu haseł (choć obecnie MD5 nie jest zalecany do tego celu z powodu problemów z bezpieczeństwem),
- w algorytmach wykrywających duplikaty, takich jak te wykorzystywane w systemach zarządzania danymi.

Algorytm MD5 działa na zasadzie iteracyjnego przetwarzania danych wejściowych podzielonych na 512-bitowe bloki, stosując złożone operacje logiczne i przesunięcia bitowe. Pomimo swojej popularności, MD5 obecnie uznawany jest za podatny na ataki kolizyjne, co oznacza, że istnieją efektywne metody pozwalające znaleźć dwa różne wejścia dające ten sam wynik hashowania.

## Cel laboratorium

Celem tego laboratorium jest:
1. Poznanie szczegółów teoretycznych algorytmu MD5.
2. Implementacja uproszczonej wersji funkcji MD5 w Pythonie i analiza jej działania.
3. Próba złamania MD5.

## Kluczowe pojęcia

- **Funkcja skrótu**: Funkcja przekształcająca dowolnie długi ciąg danych wejściowych w wynik o ustalonej długości (skrót). Funkcja skrótu jest niemożliwa do odwrócenia i powinna być odporna na kolizje.
- **Kolizja**: Przypadek, w którym dwa różne zestawy danych wejściowych generują ten sam skrót, co stanowi problem dla funkcji skrótu stosowanych w kryptografii.
- **Pseudolosowe mieszanie**: Proces generowania wartości haszujących oparty na bitowych operacjach logicznych, które wprowadzają złożoność i nieprzewidywalność.

> **Notatka:** Chociaż MD5 jest łatwy do zrozumienia i implementacji, nie jest bezpieczny do ochrony haseł ani przechowywania danych wrażliwych. Dlatego będziemy go analizować tylko w celach edukacyjnych.


# Algorytm MD5

## Krok 1 - inicjalizacja stałych

In [1]:
import math
# Stałe inicjalizacyjne dla MD5
A0, B0, C0, D0 = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476

# Stałe przesunięcia bitowe dla każdej rundy
shift_amounts = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21
]

# Tablica stałych sinusowych T
T = [int(4294967296 * abs(math.sin(i + 1))) & 0xFFFFFFFF for i in range(64)]

## Krok 2 - funkcje pomocnicze

In [2]:
# Funkcje pomocnicze
def left_rotate(x, amount):
    # Napisz funkcję rotującą binarnie w lewo słowo 32 bitowe
    return (x << amount | x >> (32 - amount)) & 0xFFFFFFFF

# Definicje funkcji F, G, H, I używanych w algorytmie
# Zwróć (x AND y) OR (NOT x AND z)
def F(x, y, z): 
    return (x & y) | (~x & z)

# Zwróć (x AND z) OR (y AND NOT z)
def G(x, y, z):
    return (x & z) | (y & ~z)

# Zwróć x XOR y XOR z
def H(x, y, z): 
    return x ^ y ^ z

# Zwróć y XOR (x OR NOT z)
def I(x, y, z):
    return y ^ (x | ~z)

## Krok 3 - padding

In [3]:
def pad_message(message):
    messageBytes = bytearray(message, 'utf-8')  # Konwertujemy wiadomość na bajty
    
    # Oblicz liczbę bitów oryginalnej wiadomości i przypisz do orig_len_in_bits
    orig_len_in_bits = len(messageBytes) * 8
    
    # Dołącz do messageBytes wartość 0x80
    messageBytes.append(0x80)
    
    # Dodawaj bajty 0x00 do messageBytes, dopóki długość messageBytes modulo 64 jest różne od 56
    while len(messageBytes) % 64 != 56:
        messageBytes.append(0x00)
    
    # Dołącz do tablicy messageBytes długość bitową orig_len_in_bits na 8 bajtach i w systemie Little Endian. Pomocna będzie metoda `to_bytes`.
    messageBytes.extend(orig_len_in_bits.to_bytes(8, 'little'))
    
    return messageBytes

## Krok 4 - główna pętla

In [4]:
def MD5(messageText):
    # wykonaj padding wiadomości
    message = pad_message(messageText)
    
    A, B, C, D = A0, B0, C0, D0
    for i in range(0, len(message), 64):
        # Blok 512-bitowy podzielony na szesnaście 32-bitowych słów:
        # Pobierz 16 razy po 4 bajty (po kolei), przekonwertuj każdą czwórkę na int w systemie Little Endian. 
        # M powinno w każdej iteracji mieć 16 liczb typu int. Pomocna może być metoda: `int.from_bytes`
        M = list()
        for j in range(i, i + 64, 4):
            M.append(int.from_bytes(message[j:j+4], 'little'))

        # Kopie bieżących wartości rejestrów
        AA, BB, CC, DD = A, B, C, D

        # 64 kroki, 4 rundy
        for j in range(64):
            if 0 <= j <= 15:
                # Przypisz do f = F(B, C, D), a do g przypisz j
                f = F(B, C, D)
                g = j
            elif 16 <= j <= 31:
                # Przypisz do f = G(B, C, D), a do g przypisz 5 razy j plus 1 modulo 16.
                f = G(B, C, D)
                g = (5 * j + 1) % 16
            elif 32 <= j <= 47:
                # Przypisz do f = H(B, C, D), a do g przypisz 3 razy j plus 5 modulo 16.
                f = H(B, C, D)
                g = (3 * j + 5) % 16
            elif 48 <= j <= 63:
                # Przypisz do f = I(B, C, D), a do g przypisz 7 razy j modulo 16.
                f = I(B, C, D)
                g = (7 * j) % 16

            # Obliczenia na rejestrach:
            # Podstaw do temp wartość D
            temp = D
            
            # Podstaw do D wartość C
            # Podstaw do C wartość B
            D = C
            C = B
            
            # Dodaj do B wartość A + f + M[g] + T[j] zrotowaną binarnie w lewo shift_amounts[j] razy
            B = (B + left_rotate((A + f + M[g] + T[j]) & 0xFFFFFFFF, shift_amounts[j])) & 0xFFFFFFFF
            
            # Podstaw do A wartość temp
            A = temp

        # Dodaj do A wartość AA
        # Dodaj do B wartość BB
        # Dodaj do C wartość CC
        # Dodaj do D wartość DD
        A = (A + AA) & 0xFFFFFFFF
        B = (B + BB) & 0xFFFFFFFF
        C = (C + CC) & 0xFFFFFFFF
        D = (D + DD) & 0xFFFFFFFF

    # Konwersja końcowa do wartości MD5 (128-bitowy hash):
    # Skonkatenuj binarnie A, B, C, D w takiej kolejności, tzn. 4 najmniej znaczące bajty to A, a 4 najbardziej znaczące to D.
    concat = (D << 96) | (C << 64) | (B << 32) | A
    
    # Przekonwertuj konkatenację do bajtów w little endian, następnie do hex i zwróć. To jest Twój skrót.
    return concat.to_bytes(16, 'little').hex()

# 0cc175b9c0f1b6a831c399e269772661
print(MD5("a"))

0cc175b9c0f1b6a831c399e269772661


## Podsumowanie

### Inicjalizacja zmiennych
MD5 rozpoczyna pracę od ustawienia czterech zmiennych (A, B, C, D), które są zainicjalizowane określonymi, stałymi wartościami. Te zmienne tworzą 128-bitowy rejestr, w którym przechowywane są aktualne wartości stanu algorytmu. To ustawienie początkowe jest zawsze takie samo i zapewnia początkowy punkt odniesienia dla obliczeń.

### Cztery rundy i funkcje mieszające (F, G, H, I)
MD5 wykonuje cztery rundy operacji, podczas których stosowane są różne funkcje mieszające (F, G, H, I) oraz operacje przesunięcia i sumowania. Każda runda przechodzi przez dane z wykorzystaniem operacji logicznych, takich jak AND, OR i XOR, aby zmieszać dane w nietrywialny sposób.

- Funkcje F, G, H, I stosują różne formy operacji bitowych, aby przemieszać bity w każdym kroku w sposób nieodwracalny. Zastosowanie tych operacji zapewnia, że niewielka zmiana w danych wejściowych prowadzi do znacznej zmiany w końcowym skrócie.
- Przesunięcia bitowe i stałe sinusoidalne (T) dodawane w każdej rundzie pomagają wprowadzić dodatkową losowość i zwiększają odporność na kolizje.

### Padding (dopełnienie) i dodanie długości
Dane wejściowe są uzupełniane (padding) do długości wielokrotności 512 bitów, co zapewnia, że blok danych ma odpowiedni rozmiar do przetwarzania. Dopełnienie polega na dodaniu bitu 1, a następnie odpowiedniej liczby zer oraz zapisaniu oryginalnej długości wiadomości. Ten krok zapewnia, że nawet jeśli dane wejściowe mają nietypową długość, algorytm może je przetworzyć.

### Dzielenie na bloki 512-bitowe i 32-bitowe słowa
Uzupełnione dane są dzielone na bloki po 512 bitów, a następnie każdy blok jest dzielony na szesnaście 32-bitowych słów. MD5 przetwarza dane w blokach, co umożliwia stosowanie iteracyjnych operacji bez względu na długość wiadomości. Każdy blok jest przetwarzany osobno, co pozwala na efektywną obróbkę dużych danych.

### Aktualizacja rejestrów
Po przetworzeniu każdego bloku 512-bitowego wartości rejestrów A, B, C i D są sumowane z ich stanem początkowym, co oznacza, że każdy blok wpływa na kolejne przetwarzane bloki. Ta operacja sprawia, że każda część wiadomości przyczynia się do ostatecznego wyniku.

### Finalizacja (Konstrukcja skrótu)
Po przetworzeniu wszystkich bloków cztery 32-bitowe wartości (A, B, C, D) są łączone, tworząc 128-bitowy skrót. Wynik jest wyrażany jako liczba szesnastkowa i stanowi końcowy skrót MD5, który identyfikuje dane wejściowe.

Wyniki możesz porównać z [tym](https://www.md5hashgenerator.com/) kalkulatorem.

In [5]:
# przetestujmy parę przypadków
testString = "Wojtek"
MD5hash = MD5(testString)
assert MD5hash == "8b8609c381cdbc6d6ff61c383b0adab0"

testString = "Szedl Sasza szosa sucha"
MD5hash = MD5(testString)
assert MD5hash == "f461e52b114047b06a9c883c0ee411f2"

testString = "Stol z powylamywanymi nogami"
MD5hash = MD5(testString)
assert MD5hash == "bfc6738b7068e4aee20fe7ec04cfcf25"

print("Jest szansa, że działa!")

Jest szansa, że działa!
